# Pose Error Project
### (Transformer-Encoder)

In [1]:
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Loading packages ...")
import os
import sys
import time
import pickle
import json

# 3rd party packages

#from tqdm import tqdm
# since we are using it in jupyter notebook
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Project modules
from options import Options
from running import setup, pipeline_factory, validate, check_progress, NEG_METRICS
from utils import utils
from datasets.data import data_factory, Normalizer
from datasets.datasplit import split_dataset
from models.ts_transformer import model_factory
from models.loss import get_loss_module
from optimizers import get_optimizer

import parser


2023-05-25 21:58:29,668 | INFO : Loading packages ...
2023-05-25 21:58:31,386 | INFO : Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-05-25 21:58:31,387 | INFO : NumExpr defaulting to 8 threads.


# Setup Configurations

In [2]:
# Setting 2 - Two Stages
# Pretrain
# text = "--output_dir ../experiments/ --comment 'poseErrorPred_pretrain' \
#         --name poseErrorPred_preTrain --records_file Regression_records.xls \
#         --data_dir ../data/Hall_LivingRoom_Pretrain/ --data_class pose \
#         --val_ratio 0.2 --epochs 30 --lr 0.00001 --optimizer RAdam --batch_size 128 \
#         --pos_encoding learnable --task regression --print_interval 1\
#         --num_layers 3  --num_heads 8 --d_model 64 --dim_feedforward 256 --gpu 2 --val_interval 1"

text = "--output_dir ../experiments/ --comment 'poseErrorPred_pretrain' \
        --name poseErrorPred_preTrain --records_file Regression_records.xls \
        --data_dir ../data/SenseTimeV4_Selective/ --data_class pose \
        --val_ratio 0.2 --epochs 30 --lr 0.00001 --optimizer RAdam --batch_size 128 \
        --pos_encoding learnable --task regression --print_interval 1\
        --num_layers 3  --num_heads 8 --d_model 64 --dim_feedforward 256 --gpu 2 --val_interval 1"

In [3]:
# Process the setting string
# Generate the config variable
input_text = text.split()
args = Options().parse(input_text)
config = setup(args)

2023-05-25 21:58:39,394 | INFO : Stored configuration file in '../experiments/poseErrorPred_preTrain_2023-05-25_21-58-39_T27'


In [4]:
config

{'config_filepath': None,
 'output_dir': '../experiments/poseErrorPred_preTrain_2023-05-25_21-58-39_T27',
 'data_dir': '../data/SenseTimeV4_Selective/',
 'load_model': None,
 'resume': False,
 'change_output': False,
 'save_all': False,
 'experiment_name': 'poseErrorPred_preTrain',
 'comment': "'poseErrorPred_pretrain'",
 'no_timestamp': False,
 'records_file': 'Regression_records.xls',
 'console': False,
 'print_interval': 1,
 'gpu': '2',
 'n_proc': -1,
 'num_workers': 0,
 'seed': None,
 'limit_size': None,
 'test_only': None,
 'data_class': 'pose',
 'labels': None,
 'test_from': None,
 'test_ratio': 0,
 'val_ratio': 0.2,
 'pattern': None,
 'val_pattern': None,
 'test_pattern': None,
 'normalization': 'standardization',
 'norm_from': None,
 'subsample_factor': None,
 'task': 'regression',
 'masking_ratio': 0.15,
 'mean_mask_length': 3,
 'mask_mode': 'separate',
 'mask_distribution': 'geometric',
 'exclude_feats': None,
 'mask_feats': [0, 1],
 'start_hint': 0.0,
 'end_hint': 0.0,
 'har

# Setup Logger

In [5]:
total_epoch_time = 0
total_eval_time = 0

total_start_time = time.time()

In [6]:
# Add file logging besides stdout
file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
logger.addHandler(file_handler)

logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

2023-05-25 21:58:52,421 | INFO : Running:
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/tianyi/.local/share/jupyter/runtime/kernel-ab7cf4ec-4309-4e29-829f-276210ab4a13.json



# Setup Training Device

In [7]:
if config['seed'] is not None:
    torch.manual_seed(config['seed'])

device = torch.device('cuda:0' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
logger.info("Using device: {}".format(device))
if device == 'cuda:0':
    logger.info("Device index: {}".format(torch.cuda.current_device()))

2023-05-25 21:58:53,661 | INFO : Using device: cuda:0


# Prepare Data

In [8]:
 # Build data
logger.info("Loading and preprocessing data ...")
data_class = data_factory[config['data_class']]
my_data = data_class(config['data_dir'], 
                     pattern=config['pattern'], 
                     n_proc=config['n_proc'], 
                     limit_size=config['limit_size'], 
                     config=config)
feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
if config['task'] == 'classification':
    validation_method = 'StratifiedShuffleSplit'
    labels = my_data.labels_df.values.flatten()
else:
    validation_method = 'ShuffleSplit'
    labels = None
    
# Modify for the pose error pred
validation_method = 'PoseErrorTimeSplit'

2023-05-25 21:58:55,358 | INFO : Loading and preprocessing data ...
2023-05-25 21:58:55,360 | INFO : Loading 69 datasets files using 32 parallel processes ...


In [9]:
feat_dim = my_data.feature_df.shape[1]
feat_dim

13

# Split dataset

In [10]:
# Split dataset
test_data = my_data
test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
val_data = my_data
val_indices = []
if config['test_pattern']:  # used if test data come from different files / file patterns
    test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
    test_indices = test_data.all_IDs
if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
    test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
    try:
        test_indices = [int(ind) for ind in test_indices]  # integer indices
    except ValueError:
        pass  # in case indices are non-integers
    logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
if config['val_pattern']:  # used if val data come from different files / file patterns
    val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
    val_indices = val_data.all_IDs

In [11]:
# Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
# Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
if config['val_ratio'] > 0:
    train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                             validation_method=validation_method,
                                                             n_splits=1,
                                                             validation_ratio=config['val_ratio'],
                                                             test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                             test_indices=test_indices,
                                                             random_seed=1337,
                                                             labels=labels)
    train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
else:
    train_indices = my_data.all_IDs
    if test_indices is None:
        test_indices = []

In [12]:
logger.info("{} \t samples may be used for training".format(len(train_indices)))
logger.info("{} \t samples will be used for validation".format(len(val_indices)))
logger.info("{} \t samples will be used for testing".format(len(test_indices)))

2023-05-25 21:59:29,046 | INFO : 87695 	 samples may be used for training
2023-05-25 21:59:29,048 | INFO : 21957 	 samples will be used for validation
2023-05-25 21:59:29,048 | INFO : 0 	 samples will be used for testing


In [13]:
with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
    try:
        json.dump({'train_indices': list(map(int, train_indices)),
                   'val_indices': list(map(int, val_indices)),
                   'test_indices': list(map(int, test_indices))}, f, indent=4)
    except ValueError:  # in case indices are non-integers
        json.dump({'train_indices': list(train_indices),
                   'val_indices': list(val_indices),
                   'test_indices': list(test_indices)}, f, indent=4)


In [14]:
# Pre-process features
normalizer = None
if config['norm_from']:
    with open(config['norm_from'], 'rb') as f:
        norm_dict = pickle.load(f)
    normalizer = Normalizer(**norm_dict)
elif config['normalization'] is not None:
    normalizer = Normalizer(config['normalization'])
    my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
    if not config['normalization'].startswith('per_sample'):
        # get normalizing values from training set and store for future use
        norm_dict = normalizer.__dict__
        with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
            pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
if normalizer is not None:
    if len(val_indices):
        val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
    if len(test_indices):
        test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])


# Create model

In [15]:
# Create model
logger.info("Creating model ...")
model = model_factory(config, my_data)

if config['freeze']:
    for name, param in model.named_parameters():
        if name.startswith('output_layer'):
            param.requires_grad = True
        else:
            param.requires_grad = False

logger.info("Model:\n{}".format(model))
logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))


2023-05-25 21:59:39,108 | INFO : Creating model ...
2023-05-25 21:59:39,132 | INFO : Model:
TSTransformerEncoderClassiregressor(
  (project_inp): Linear(in_features=13, out_features=64, bias=True)
  (pos_enc): LearnablePositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerBatchNormEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dro

# Initialize optimizer

In [16]:
# Initialize optimizer
if config['global_reg']:
    weight_decay = config['l2_reg']
    output_reg = None
else:
    weight_decay = 0
    output_reg = config['l2_reg']

optim_class = get_optimizer(config['optimizer'])
optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

start_epoch = 0
lr_step = 0  # current step index of `lr_step`
lr = config['lr']  # current learning step
# Load model and optimizer state
if args.load_model:
    model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                     config['change_output'],
                                                     config['lr'],
                                                     config['lr_step'],
                                                     config['lr_factor'])
model.to(device)

loss_module = get_loss_module(config)

In [17]:
if config['test_only'] == 'testset':  # Only evaluate and skip training
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    test_dataset = dataset_class(test_data, test_indices)

    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=config['batch_size'],
                             shuffle=False,
                             num_workers=config['num_workers'],
                             pin_memory=True,
                             collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
    test_evaluator = runner_class(model, test_loader, device, loss_module,
                                        print_interval=config['print_interval'], console=config['console'])
    aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
    print_str = 'Test Summary: '
    for k, v in aggr_metrics_test.items():
        print_str += '{}: {:8f} | '.format(k, v)
    logger.info(print_str)
    #return

In [18]:
# Initialize data generators
if config['test_only'] != 'testset':  # Only evaluate and skip training
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    val_dataset = dataset_class(val_data, val_indices)

    val_loader = DataLoader(dataset=val_dataset,
                            batch_size=config['batch_size'],
                            shuffle=False,
                            num_workers=config['num_workers'],
                            pin_memory=True,
                            collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    train_dataset = dataset_class(my_data, train_indices)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=config['batch_size'],
                              shuffle=True,
                              num_workers=config['num_workers'],
                              pin_memory=True,
                              collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    trainer = runner_class(model, train_loader, device, loss_module, optimizer, l2_reg=output_reg,
                                 print_interval=config['print_interval'], console=config['console'])
    val_evaluator = runner_class(model, val_loader, device, loss_module,
                                       print_interval=config['print_interval'], console=config['console'])

    tensorboard_writer = SummaryWriter(config['tensorboard_dir'])

    best_value = 1e16 if config['key_metric'] in NEG_METRICS else -1e16  # initialize with +inf or -inf depending on key metric
    metrics = []  # (for validation) list of lists: for each epoch, stores metrics like loss, ...
    best_metrics = {}

In [19]:
'''
print(config["batch_size"])
for batch in train_loader:
    X, targets, padding_masks, IDs = batch
    print(X.shape)
    print(X[0])
    print(X.dtype)
    print("-"*20)
    print(targets.shape)
    print(targets[0])
    print(targets.dtype)
    print("-"*20)
    print(padding_masks.shape)
    print(padding_masks[0])
    print("-"*20)
    print(IDs)
    break
'''

'\nprint(config["batch_size"])\nfor batch in train_loader:\n    X, targets, padding_masks, IDs = batch\n    print(X.shape)\n    print(X[0])\n    print(X.dtype)\n    print("-"*20)\n    print(targets.shape)\n    print(targets[0])\n    print(targets.dtype)\n    print("-"*20)\n    print(padding_masks.shape)\n    print(padding_masks[0])\n    print("-"*20)\n    print(IDs)\n    break\n'

# Evaluate on validation before training

In [20]:
aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config, best_metrics,
                                                      best_value, epoch=0)
metrics_names, metrics_values = zip(*aggr_metrics_val.items())
metrics.append(list(metrics_values))

2023-05-25 21:59:47,070 | INFO : Evaluating on validation set ...


Evaluating Epoch 0   0.0% | batch:         0 of       172	|	loss: 18562.2
Evaluating Epoch 0   0.6% | batch:         1 of       172	|	loss: 16663.2
Evaluating Epoch 0   1.2% | batch:         2 of       172	|	loss: 17883.6
Evaluating Epoch 0   1.7% | batch:         3 of       172	|	loss: 10908.9
Evaluating Epoch 0   2.3% | batch:         4 of       172	|	loss: 17692.3
Evaluating Epoch 0   2.9% | batch:         5 of       172	|	loss: 5672.17
Evaluating Epoch 0   3.5% | batch:         6 of       172	|	loss: 21810.5
Evaluating Epoch 0   4.1% | batch:         7 of       172	|	loss: 10053.1
Evaluating Epoch 0   4.7% | batch:         8 of       172	|	loss: 15211.4
Evaluating Epoch 0   5.2% | batch:         9 of       172	|	loss: 15051.4
Evaluating Epoch 0   5.8% | batch:        10 of       172	|	loss: 11370.3
Evaluating Epoch 0   6.4% | batch:        11 of       172	|	loss: 19053.3
Evaluating Epoch 0   7.0% | batch:        12 of       172	|	loss: 5556.19
Evaluating Epoch 0   7.6% | batch:    

Evaluating Epoch 0  67.4% | batch:       116 of       172	|	loss: 27538.2
Evaluating Epoch 0  68.0% | batch:       117 of       172	|	loss: 17829.3
Evaluating Epoch 0  68.6% | batch:       118 of       172	|	loss: 12735.5
Evaluating Epoch 0  69.2% | batch:       119 of       172	|	loss: 34265.8
Evaluating Epoch 0  69.8% | batch:       120 of       172	|	loss: 23691.9
Evaluating Epoch 0  70.3% | batch:       121 of       172	|	loss: 25640.1
Evaluating Epoch 0  70.9% | batch:       122 of       172	|	loss: 33486.2
Evaluating Epoch 0  71.5% | batch:       123 of       172	|	loss: 48895.4
Evaluating Epoch 0  72.1% | batch:       124 of       172	|	loss: 225596
Evaluating Epoch 0  72.7% | batch:       125 of       172	|	loss: 43693.4
Evaluating Epoch 0  73.3% | batch:       126 of       172	|	loss: 15405.9
Evaluating Epoch 0  73.8% | batch:       127 of       172	|	loss: 9761.76
Evaluating Epoch 0  74.4% | batch:       128 of       172	|	loss: 34083.4
Evaluating Epoch 0  75.0% | batch:     

2023-05-25 21:59:52,783 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 5.713092803955078 seconds

2023-05-25 21:59:52,784 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 5.713092803955078 seconds
2023-05-25 21:59:52,784 | INFO : Avg batch val. time: 0.033215655836948126 seconds
2023-05-25 21:59:52,785 | INFO : Avg sample val. time: 0.00026019459871362566 seconds
2023-05-25 21:59:52,785 | INFO : Epoch 0 Validation Summary: epoch: 0.000000 | loss: 10738.389361 | 


Evaluating Epoch 0  99.4% | batch:       171 of       172	|	loss: 10355



/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


# Starting training...

In [21]:
logger.info('Starting training...')
for epoch in tqdm(range(start_epoch + 1, config["epochs"] + 1), desc='Training Epoch', leave=False):
    mark = epoch if config['save_all'] else 'last'
    epoch_start_time = time.time()
    # Training
    aggr_metrics_train = trainer.train_epoch(epoch)  # dictionary of aggregate epoch metrics
    epoch_runtime = time.time() - epoch_start_time
    print()
    print_str = 'Epoch {} Training Summary: '.format(epoch)
    for k, v in aggr_metrics_train.items():
        tensorboard_writer.add_scalar('{}/train'.format(k), v, epoch)
        print_str += '{}: {:8f} | '.format(k, v)
    logger.info(print_str)
    logger.info("Epoch runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(epoch_runtime)))
    total_epoch_time += epoch_runtime
    avg_epoch_time = total_epoch_time / (epoch - start_epoch)
    avg_batch_time = avg_epoch_time / len(train_loader)
    avg_sample_time = avg_epoch_time / len(train_dataset)
    logger.info("Avg epoch train. time: {} hours, {} minutes, {} seconds".format(*utils.readable_time(avg_epoch_time)))
    logger.info("Avg batch train. time: {} seconds".format(avg_batch_time))
    logger.info("Avg sample train. time: {} seconds".format(avg_sample_time))

    # evaluate if first or last epoch or at specified interval
    if (epoch == config["epochs"]) or (epoch == start_epoch + 1) or (epoch % config['val_interval'] == 0):
        aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config,
                                                              best_metrics, best_value, epoch)
        metrics_names, metrics_values = zip(*aggr_metrics_val.items())
        metrics.append(list(metrics_values))

    utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(mark)), epoch, model, optimizer)

    # Learning rate scheduling
    if epoch == config['lr_step'][lr_step]:
        utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(epoch)), epoch, model, optimizer)
        lr = lr * config['lr_factor'][lr_step]
        if lr_step < len(config['lr_step']) - 1:  # so that this index does not get out of bounds
            lr_step += 1
        logger.info('Learning rate updated to: ', lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    # Difficulty scheduling
    if config['harden'] and check_progress(epoch):
        train_loader.dataset.update()
        val_loader.dataset.update()


2023-05-25 21:59:52,846 | INFO : Starting training...


Training Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

/home/tianyi/Documents/mvts_transformer/src/optimizers.py:69: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Training Epoch 1   0.0% | batch:         0 of       686	|	loss: 118.869
Training Epoch 1   0.1% | batch:         1 of       686	|	loss: 114.254
Training Epoch 1   0.3% | batch:         2 of       686	|	loss: 145.515
Training Epoch 1   0.4% | batch:         3 of       686	|	loss: 112.183
Training Epoch 1   0.6% | batch:         4 of       686	|	loss: 139.662
Training Epoch 1   0.7% | batch:         5 of       686	|	loss: 192.13
Training Epoch 1   0.9% | batch:         6 of       686	|	loss: 100.377
Training Epoch 1   1.0% | batch:         7 of       686	|	loss: 121.954
Training Epoch 1   1.2% | batch:         8 of       686	|	loss: 125.526
Training Epoch 1   1.3% | batch:         9 of       686	|	loss: 126.244
Training Epoch 1   1.5% | batch:        10 of       686	|	loss: 143.211
Training Epoch 1   1.6% | batch:        11 of       686	|	loss: 141.221
Training Epoch 1   1.7% | batch:        12 of       686	|	loss: 97.1591
Training Epoch 1   1.9% | batch:        13 of       686	|	loss: 4

Training Epoch 1  16.6% | batch:       114 of       686	|	loss: 69.7103
Training Epoch 1  16.8% | batch:       115 of       686	|	loss: 100.522
Training Epoch 1  16.9% | batch:       116 of       686	|	loss: 70.4222
Training Epoch 1  17.1% | batch:       117 of       686	|	loss: 182.101
Training Epoch 1  17.2% | batch:       118 of       686	|	loss: 116.089
Training Epoch 1  17.3% | batch:       119 of       686	|	loss: 80.3607
Training Epoch 1  17.5% | batch:       120 of       686	|	loss: 89.0706
Training Epoch 1  17.6% | batch:       121 of       686	|	loss: 80.7908
Training Epoch 1  17.8% | batch:       122 of       686	|	loss: 89.8916
Training Epoch 1  17.9% | batch:       123 of       686	|	loss: 89.1305
Training Epoch 1  18.1% | batch:       124 of       686	|	loss: 84.3103
Training Epoch 1  18.2% | batch:       125 of       686	|	loss: 118.967
Training Epoch 1  18.4% | batch:       126 of       686	|	loss: 91.0643
Training Epoch 1  18.5% | batch:       127 of       686	|	loss: 

...

2023-05-25 22:13:48,133 | INFO : Epoch 30 Training Summary: epoch: 30.000000 | loss: 0.681944 | 
2023-05-25 22:13:48,135 | INFO : Epoch runtime: 0.0 hours, 0.0 minutes, 26.0944561958313 seconds

2023-05-25 22:13:48,138 | INFO : Avg epoch train. time: 0.0 hours, 0.0 minutes, 23.924308451016746 seconds
2023-05-25 22:13:48,138 | INFO : Avg batch train. time: 0.03487508520556377 seconds
2023-05-25 22:13:48,138 | INFO : Avg sample train. time: 0.0002728126854554621 seconds
2023-05-25 22:13:48,139 | INFO : Evaluating on validation set ...


Training Epoch 30  99.6% | batch:       683 of       686	|	loss: 0.560604
Training Epoch 30  99.7% | batch:       684 of       686	|	loss: 0.616593
Training Epoch 30  99.9% | batch:       685 of       686	|	loss: 0.70871

Evaluating Epoch 30   0.0% | batch:         0 of       172	|	loss: 1.41099
Evaluating Epoch 30   0.6% | batch:         1 of       172	|	loss: 1.38584
Evaluating Epoch 30   1.2% | batch:         2 of       172	|	loss: 0.664972
Evaluating Epoch 30   1.7% | batch:         3 of       172	|	loss: 1.94126
Evaluating Epoch 30   2.3% | batch:         4 of       172	|	loss: 1.12841
Evaluating Epoch 30   2.9% | batch:         5 of       172	|	loss: 1.142
Evaluating Epoch 30   3.5% | batch:         6 of       172	|	loss: 1.23732
Evaluating Epoch 30   4.1% | batch:         7 of       172	|	loss: 2.38596
Evaluating Epoch 30   4.7% | batch:         8 of       172	|	loss: 0.714465
Evaluating Epoch 30   5.2% | batch:         9 of       172	|	loss: 1.5137
Evaluating Epoch 30   5.8% | 

Evaluating Epoch 30  61.6% | batch:       106 of       172	|	loss: 0.427601
Evaluating Epoch 30  62.2% | batch:       107 of       172	|	loss: 0.822955
Evaluating Epoch 30  62.8% | batch:       108 of       172	|	loss: 0.418226
Evaluating Epoch 30  63.4% | batch:       109 of       172	|	loss: 0.600233
Evaluating Epoch 30  64.0% | batch:       110 of       172	|	loss: 0.648826
Evaluating Epoch 30  64.5% | batch:       111 of       172	|	loss: 0.589019
Evaluating Epoch 30  65.1% | batch:       112 of       172	|	loss: 0.776994
Evaluating Epoch 30  65.7% | batch:       113 of       172	|	loss: 0.59079
Evaluating Epoch 30  66.3% | batch:       114 of       172	|	loss: 0.619127
Evaluating Epoch 30  66.9% | batch:       115 of       172	|	loss: 0.259906
Evaluating Epoch 30  67.4% | batch:       116 of       172	|	loss: 0.186892
Evaluating Epoch 30  68.0% | batch:       117 of       172	|	loss: 0.191307
Evaluating Epoch 30  68.6% | batch:       118 of       172	|	loss: 0.164126
Evaluating Ep

2023-05-25 22:13:52,052 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 3.9132566452026367 seconds

2023-05-25 22:13:52,056 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 4.032936872974519 seconds
2023-05-25 22:13:52,057 | INFO : Avg batch val. time: 0.023447307401014642 seconds
2023-05-25 22:13:52,057 | INFO : Avg sample val. time: 0.00018367431219995986 seconds
2023-05-25 22:13:52,058 | INFO : Epoch 30 Validation Summary: epoch: 30.000000 | loss: 1.301544 | 


Evaluating Epoch 30  94.8% | batch:       163 of       172	|	loss: 0.388941
Evaluating Epoch 30  95.3% | batch:       164 of       172	|	loss: 0.28844
Evaluating Epoch 30  95.9% | batch:       165 of       172	|	loss: 0.31049
Evaluating Epoch 30  96.5% | batch:       166 of       172	|	loss: 0.19876
Evaluating Epoch 30  97.1% | batch:       167 of       172	|	loss: 0.254291
Evaluating Epoch 30  97.7% | batch:       168 of       172	|	loss: 0.303571
Evaluating Epoch 30  98.3% | batch:       169 of       172	|	loss: 0.126697
Evaluating Epoch 30  98.8% | batch:       170 of       172	|	loss: 0.217496
Evaluating Epoch 30  99.4% | batch:       171 of       172	|	loss: 0.37783



In [22]:
# Export evolution of metrics over epochs
header = metrics_names
metrics_filepath = os.path.join(config["output_dir"], "metrics_" + config["experiment_name"] + ".xls")
book = utils.export_performance_metrics(metrics_filepath, metrics, header, sheet_name="metrics")

# Export record metrics to a file accumulating records from all experiments
utils.register_record(config["records_file"], config["initial_timestamp"], config["experiment_name"],
                      best_metrics, aggr_metrics_val, comment=config['comment'])

logger.info('Best {} was {}. Other metrics: {}'.format(config['key_metric'], best_value, best_metrics))
logger.info('All Done!')

total_runtime = time.time() - total_start_time
logger.info("Total runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(total_runtime)))

#return best_value
print(best_value)

2023-05-25 22:13:52,079 | INFO : Exported per epoch performance metrics in '../experiments/poseErrorPred_preTrain_2023-05-25_21-58-39_T27/metrics_poseErrorPred_preTrain.xls'
2023-05-25 22:13:52,101 | INFO : Exported performance record to 'Regression_records.xls'
2023-05-25 22:13:52,103 | INFO : Best loss was 1.1690099435344052. Other metrics: OrderedDict([('epoch', 28), ('loss', 1.1690099435344052)])
2023-05-25 22:13:52,104 | INFO : All Done!
2023-05-25 22:13:52,105 | INFO : Total runtime: 0.0 hours, 14.0 minutes, 59.834444522857666 seconds



1.1690099435344052


# Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
base_path = "../experiments/"
experiment = 'poseErrorPred_finetuned_2023-05-16_12-04-46_ia1'
file_path = '/predictions/best_predictions.npz'
total_path = base_path + experiment + file_path
total_path
#total_path = config['pred_dir']

In [ ]:
total_path = config['pred_dir'] + '/best_predictions.npz'
total_path

In [ ]:
config['pred_dir']

In [ ]:
config['output_dir']

In [ ]:
pred = np.load(total_path, allow_pickle=True)
pred.files

In [ ]:
y = np.concatenate(pred["targets"], axis=0)
y_pred = np.concatenate(pred["predictions"], axis=0)
IDs = np.concatenate(pred["IDs"], axis=0)
y_pred.shape

In [ ]:
def get_mape(y, y_pred):
    err = y - y_pred
    return np.mean(np.abs(err)/y)

def get_mse(y, y_pred):
    err = y - y_pred
    return np.mean(np.square(err))

In [ ]:
get_mse(y, y_pred)

In [ ]:
get_mape(y, y_pred)

In [ ]:
def visualize_output(y, y_pred, title=' '):
    fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(14, 6))
    fig.tight_layout(pad=3.0)
    
    ax0.plot(y, color='g', label='truth')
    ax0.set_title(title)
    ax0.set_xlabel('Step')
    ax0.set_ylabel('Error')
    ax0.grid()
    ax0.legend()
    
    ax1.plot(y, color='g', label='truth')
    ax1.plot(y_pred, color='b', alpha=0.7, label='predict')
    mse = get_mse(y, y_pred)
    mape = get_mape(y, y_pred)
    ax1.set_title(title+"- mse: {:.5f} | mape: {:.5f}".format(mse, mape))
    ax1.set_xlabel('Step')
    ax1.set_ylabel('Error')
    ax1.grid()
    ax1.legend()
    plt.show()

In [ ]:
visualize_output(y, y_pred, title='Prediction on SenseTime Dataset')

In [ ]:
IDs[10000]